### GOAL- goal is to use data from a mental health survey to explore factors that may cause individuals to experience depression.

## Notes:
    A number of data artifacts have been left in the synthetic dataset.
    This is not a particularly difficult dataset to model. It may be interesting to focus on different ways to visualize the dataset.

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

In [3]:
def cleaning(df):
    df.drop(columns=['City','id','Name','Profession','Family History of Mental Illness','Degree','Academic Pressure',	'Work Pressure',	'CGPA'	,'Study Satisfaction'	,'Job Satisfaction'],inplace=True)
    df['age_binned']=pd.cut(df['Age'], bins=[18, 25, 35, 45, 55, 65], 
                            labels=['20s', '30s', '40s', '50s', '60s'], 
                            include_lowest=True)
    df.drop(columns=['Age'],inplace=True)
    return df


In [4]:
cleaning(df_train)
cleaning(df_test)

df_test = df_test[df_test['Dietary Habits'].isin(['Moderate', 'Unhealthy', 'Healthy']) == True]
df_test = df_test[df_test['Sleep Duration'].isin(['Less than 5 hours', '7-8 hours', 'More than 8 hours']) == True]

df_train = df_train[df_train['Dietary Habits'].isin(['Moderate', 'Unhealthy', 'Healthy']) == True]
df_train = df_train[df_train['Sleep Duration'].isin(['Less than 5 hours', '7-8 hours', 'More than 8 hours']) == True]

In [ ]:
target = 'Depression'

scaling_columns=['Work/Study Hours','Financial Stress']

one_hot_columns = [
    "age_binned","Gender", "Working Professional or Student"
]

label_columns = [
     "Dietary Habits", "Have you ever had suicidal thoughts ?"
]

In [7]:
df_train.head()

,Gender,Working Professional or Student,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression,age_binned
0,Female,Working Professional,More than 8 hours,Healthy,No,1.0,2.0,0,50s
1,Male,Working Professional,Less than 5 hours,Unhealthy,Yes,7.0,3.0,1,30s
3,Male,Working Professional,Less than 5 hours,Moderate,Yes,10.0,1.0,1,20s
6,Male,Working Professional,7-8 hours,Moderate,No,6.0,2.0,0,50s
7,Male,Working Professional,7-8 hours,Unhealthy,No,10.0,3.0,0,40s


In [8]:
df_test.head()

,Gender,Working Professional or Student,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,age_binned
0,Male,Working Professional,Less than 5 hours,Moderate,No,9.0,3.0,50s
1,Female,Working Professional,Less than 5 hours,Moderate,No,6.0,4.0,60s
2,Male,Working Professional,7-8 hours,Moderate,Yes,12.0,4.0,50s
3,Female,Student,More than 8 hours,Moderate,Yes,10.0,4.0,20s
4,Male,Working Professional,7-8 hours,Moderate,Yes,3.0,4.0,50s


## Cleaning done, now scaling and label 

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import accuracy_score

### Creating a custom class for data preprocessor 

In [ ]:
class DataP